# Calculate the open and close year for each school

In [2]:
#"final.csv" is a file merged by files that contain all the public schools in U.S. from 1998 - 2016
#I'm trying to get the openYear and closeYear for each school and make a new big file.
import csv
import pandas
from os import listdir
import numpy as np
from operator import eq

In [60]:
# Read the source file
df = pandas.read_csv("final.csv")

/Users/shiji/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,2,3,4,6,7,9,11,12,13,14,15,16,20,21,22,23,24,28,29,31,32,33,34,35,37,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,67,269,270,271,272,352,353,354,355,356,357,358,359,361,362,363,365,366,368,369,371,372,374,375,377,378,380,381,383,384,386,387,389,390,392,393,395,396,398,399,401,402,403,404,406,408,409,411,413,414,416,418,419,421,422,423,425) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
#make two series to store the newly created columns
length = df.shape[0]

#initialize the two columns
YEAR_OPENED = ["NA" for i in range(length)]
YEAR_CLOSED = [None for i in range(length)]

#the names of all the status col
status = df[['STATUS98', 'STATUS99', 'STATUS00', 'STATUS01', 'STATUS02', 'STATUS03',\
             'STATUS04', 'STATUS05', 'STATUS06', 'STATUS07', 'STATUS08', 'STATUS09',\
             'STATUS10', 'STATUS11', 'STATUS12', 'STATUS13', 'SY_STATUS', 'SY_STATUS15', 'SY_STATUS16']]

#status.to_csv("status.csv", index = False)

#dictionary of (status col : corresponding year)
cols = {'STATUS98' : '1998', 'STATUS99' : '1999', 'STATUS00' : '2000', 'STATUS01' : '2001',\
        'STATUS02' : '2002', 'STATUS03' : '2003', 'STATUS04' : '2004', 'STATUS05' : '2005',\
        'STATUS06' : '2006', 'STATUS07' : '2007', 'STATUS08' : '2008', 'STATUS09' : '2009',\
        'STATUS10' : '2010', 'STATUS11' : '2011', 'STATUS12' : '2012', 'STATUS13' : '2013',\
        'SY_STATUS' : '2014', 'SY_STATUS15' : '2015', 'SY_STATUS16' : '2016'}

#status as a list
b = ['STATUS98', 'STATUS99', 'STATUS00', 'STATUS01', 'STATUS02', 'STATUS03', 'STATUS04',\
     'STATUS05', 'STATUS06', 'STATUS07', 'STATUS08', 'STATUS09', 'STATUS10', 'STATUS11',\
     'STATUS12', 'STATUS13', 'SY_STATUS', 'SY_STATUS15', 'SY_STATUS16']

In [62]:
#check all the years before the year we are looking. 
#if there is a number(except for -1, 5, 6, 7) appears, return False.
#else return True.

def checkPrevYears(statusArr, currYear):
    allNeg1 = True
    for prevStat in statusArr[:b.index(currYear)]:
        if prevStat != -1 and prevStat != 5 and prevStat != 6 and prevStat != 7:
            allNeg1 = False
    return allNeg1

In [63]:
#check all the years after the year we are looking.
#if there is a 2 or 6, return True. else return False

def checkAfterYears(statusArr, currYear):
    is2or6 = False
    for afterStat in statusArr[b.index(currYear):]:
        if afterStat == 2 or afterStat == 6:
            is2or6 = True
    return is2or6

In [64]:
def checkAfter6(statusArr, currYear):
    anythingAfter = False
    for afterStat in statusArr[b.index(currYear):]:
        if afterStat != None:
            anythingAfter = True
    return anythingAfter

In [65]:
#check all the years for a certain school.
#if there exist a 2 or 6, return True; else return False;

def checkAllYears(statusArr):
    is2or6 = False
    for stat in statusArr:
        if stat == 2 or stat == 6:
            is2or6 = True
    return is2or6

In [66]:
#main algorithm for a certain interval, inclusive.
#go through each row
YEAR_OPENED = ["NA" for i in range(length)]
YEAR_CLOSED = ["NA" for i in range(length)]
for index, row in status.iterrows():
    #only calculate for the input interval
    if index >= 0 and index < df.shape[0]:
        statusArr = [] #a list used to store all status for a year

        #putting things in statusArr
        for i in range(len(cols)):
            if np.isnan(row[b[i]]):
                statusArr += [-1] #-1 if there is a nan in certain status
            else:
                statusArr += [int(row[b[i]])]

        #main logic         
        for col in cols:    
            stat = -1
            if(not np.isnan(row[col])):
                stat = int(row[col])

            #if there is a 2, the close_year should be this year
            if(stat == 2):
                YEAR_CLOSED[index] = cols.get(col)

            #if 1,3 or 8, checkPreYears, if no previous status, open_year should be this year
            if(stat == 1 or stat == 3 or stat == 8):
                if checkPrevYears(statusArr, col):
                    YEAR_OPENED[index] = cols.get(col)

            #if 4, checkPrevYears, if no previous status, open_year should be the year before this year        
            if(stat == 4):
                if checkPrevYears(statusArr, col):
                    YEAR_OPENED[index] = cols.get(b[(b.index(col) - 1)])

            #if 8, checkLaterYear, if nothing after, year_closed should be this year
            if(stat == 8):
                if not checkAfterYears(statusArr, col):
                    YEAR_CLOSED[index] = 'NA'
            
            #if nothing after 6, year_closed should be this year
            if(stat == 6):
                if not checkAfter6(statusArr, col):
                    YEAR_CLOSED[index] = cols.get(col)

        #if never 2 or 6, year_closed is 'NA'
        if not checkAllYears(statusArr):
            YEAR_CLOSED[index] = 'NA'

In [67]:
# This is used for testing only
for index in range(0, df.shape[0]):
    if YEAR_CLOSED[index] == None or YEAR_OPENED[index] == None:
        print(index,"|", YEAR_OPENED[index],"|", YEAR_CLOSED[index])

In [68]:
# Create two dataframe for the mappings
label = ['YEAR_OPENED']
label2 = ['YEAR_CLOSED']
dfOpened = pandas.DataFrame(YEAR_OPENED, columns = label)
dfClosed = pandas.DataFrame(YEAR_CLOSED, columns = label2)

In [69]:
print(dfOpened)

       YEAR_OPENED
0             1998
1             1998
2             2003
3             2003
4             1998
5             1998
6             2006
7             1998
8             1998
9             1998
10            1998
11            1998
12            2009
13            1998
14            1998
15            1998
16            1998
17            1998
18            1998
19            1998
20            1998
21            2016
22            2016
23            1999
24            2001
25            2003
26            1998
27            1998
28            2012
29            1998
...            ...
136774        2016
136775        2016
136776        2016
136777        2016
136778        2016
136779        2016
136780        2016
136781        2016
136782        2016
136783        2016
136784        2016
136785        2016
136786        2016
136787        2016
136788        2016
136789        2016
136790        2016
136791        2016
136792        2016
136793        2016
136794      

In [70]:
print(dfClosed)

       YEAR_CLOSED
0               NA
1             2014
2               NA
3               NA
4               NA
5               NA
6               NA
7               NA
8               NA
9               NA
10              NA
11              NA
12              NA
13              NA
14              NA
15              NA
16              NA
17              NA
18              NA
19              NA
20              NA
21              NA
22              NA
23              NA
24              NA
25              NA
26              NA
27              NA
28              NA
29              NA
...            ...
136774          NA
136775          NA
136776          NA
136777          NA
136778          NA
136779          NA
136780          NA
136781          NA
136782          NA
136783          NA
136784          NA
136785          NA
136786          NA
136787          NA
136788          NA
136789          NA
136790          NA
136791          NA
136792          NA
136793          NA
136794      

In [71]:
print(df)

         SURVYEAR  FIPST STABR STATENAME                          SEANAME  \
0       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
1       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
2       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
3       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
4       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
5       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
6       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
7       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
8       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
9       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
10      2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
11      2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   

In [72]:
# Concatenating the two mapping dataframe to our source
# Here we use concat instead of merge because the sequence of NCESSCH is preserved
result = pandas.concat([df, dfOpened, dfClosed], axis=1)

In [73]:
print(result)

         SURVYEAR  FIPST STABR STATENAME                          SEANAME  \
0       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
1       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
2       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
3       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
4       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
5       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
6       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
7       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
8       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
9       2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
10      2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   
11      2014-2015    1.0    AL   ALABAMA  Alabama Department Of Education   

In [74]:
#Turn the merged dataframe to csv file
result.to_csv("openAndClose.csv", index = False)

In [75]:
#check duplicates
print(len(result['NCESSCH']) == len(result['NCESSCH'].unique()), len(result['NCESSCH']), len(result['NCESSCH'].unique()), " unique for file 1")

True 136804 136804  unique for file 1
